<a href="https://colab.research.google.com/github/danielfrgs/LCD-CC-2022-I/blob/main/Clase_4_Class_Pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import multiprocessing
# Crear un proceso que hereda de clase

class MyProcess(multiprocessing.Process):
    def __init__(self):
        super(MyProcess, self).__init__()

    def run(self):
        print("Child Process PID: {}".format(multiprocessing.current_process().pid))

def main():
    print("Main Process PID: {}".format(multiprocessing.current_process().pid))
    myProcess = MyProcess()
    myProcess.start()
    myProcess.join()

main()

# Comunicación de pipe

from multiprocessing import Process, Pipe

def worker(conn):
    print(conn.recv())
    time.sleep(1.8)
    conn.send("sent from child process")
    conn.close()

conn1, conn2 = multiprocessing.Pipe()
process = multiprocessing.Process(target=worker, args=(conn2,))
process.start()

conn1.send("sent from main process")
print(conn1.recv())
process.join()

# Otro ejemplo de PIPE
# Aquí solo es enre dos procesos

import multiprocessing

from multiprocessing import Process, Pipe

nombres = ["Carlos", "Renata", "Rebeca", "Sandra", "END"]

def send_msgs(conn, msgs):
    for msg in msgs:
        conn.send(msg)
    conn.close()

def recv_msgs(conn):

    while 1:
        msg = conn.recv()
        if msg == "END":
            break
        print(msg)

parent_conn, child_conn = multiprocessing.Pipe()

p1 = multiprocessing.Process(target=send_msgs, args=(parent_conn, nombres))
p2 = multiprocessing.Process(target=recv_msgs, args=(child_conn,))


p1.start()
p2.start()
p1.join()
p2.join()


Main Process PID: 64
Child Process PID: 106
sent from main process


Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-2-6318868e63e6>", line 25, in worker
    time.sleep(1.8)
NameError: name 'time' is not defined


KeyboardInterrupt: ignored

In [3]:

# Manejo de sistema operativo


import os, sys
import multiprocessing

class ChildProcess(multiprocessing.Process):

    def __init__(self, pipein):
        super(ChildProcess, self).__init__()
        self.pipein = pipein

    def run(self):
        print("Intentando iniciar el pipe")
        self.pipein = os.fdopen(self.pipein, 'w')
        self.pipein.write("Mi código es MX-317")
        self.pipein.close()


def main():
    pipeout, pipein = os.pipe()

    child = ChildProcess(pipein)
    child.start()
    child.join()

    os.close(pipein)
    pipeout = os.fdopen(pipeout)

    pipeContent = pipeout.read()
    print("Pipe: {}".format(pipeContent))

if __name__== '__main__':
    main()


Intentando iniciar el pipe
Pipe: Mi código es MX-317


In [4]:
# Colas

import multiprocessing

class MyWorker():
    def __init__(self, x):
        self.x = x

    def process(self):
        pname = multiprocessing.current_process().name
        print('Starting process %s for number %i...' % (pname, self.x))

        
def work(q):
    worker = q.get()
    worker.process()

if __name__ == '__main__':
    my_queue = multiprocessing.Queue()

    p = multiprocessing.Process(target=work, args=(my_queue,))
    p.start()

    my_queue.put(MyWorker(1))

    my_queue.close()
    my_queue.join_thread()   ## investigar ...
    p.join()

    print('Done.')

Starting process Process-4 for number 1...
Done.
